## 빅데이터 실습

### 다나와 무선청소기 상품분석

#### 데이터 수집

##### OpenAPI
- 한국관광공사 관광빅데이터 정보서비 GW
  - 기본URL : http://apis.data.go.kr/B551011/DataLabService/metcoRegnVisitrDDList
  - 서비스키 : qD9EPBoeW3jjfktf3X%2Ftr3syHNjWfOzloUDZ5WfJwKtsyVCRzFy2v4OfNffjH5ulAtfuFrINMfhraiEFLeVmNQ%3D%3D (encoding 키를 사용)
  - 피라미터(?뒤)
    - serviceKey = 본인서비스키
    - numOfRows = 한페이지나타낼 결과수(10기본 60으로 하면 하루치 전부다 가져올 수 있음)
    - pageNo = 페이지번호(1)
    - MobileOs = IOS, AND, WIN, ETC(기타)
    - MobileApp = AppTest(테스트)
    - startYmd = 시작일자
    - endYmd = 종료일자

In [ ]:
import requests
import json
import xmltodict
import pandas as pd

##### 추가모듈
- XML을 딕셔너리로 한번 바꿔주는 모듈
- xmltodic
- !pip install xmltodiict

In [ ]:
!pip install xmltodict

In [ ]:
serviceKey = r'qD9EPBoeW3jjfktf3X%2Ftr3syHNjWfOzloUDZ5WfJwKtsyVCRzFy2v4OfNffjH5ulAtfuFrINMfhraiEFLeVmNQ%3D%3D'

In [ ]:
ymd = '20210202'

In [ ]:
baseUrl = 'http://apis.data.go.kr/B551011/DataLabService/metcoRegnVisitrDDList?'
parameter = f'serviceKey={serviceKey}&numOfRows=60&pageNo=1&MobileOS=ETC&MobileApp=AppTest&' + \
            f'startYmd={ymd}&endYmd={ymd}'
totalUrl = baseUrl + parameter
totalUrl

In [ ]:
# 실행 테스트
# res.status_code : 200(OK) or 404/500 | res.text(str) / res.content(byte)
res = requests.get(totalUrl)
res.status_code
res.text

In [ ]:
# openAPI로 넘어온 text(xml타입)
## xmltodict.parse() xml문서를 파이썬 dictionary타입 변환
## josn.dumps() dictionary를 json 타입으로 변환
json.dumps(xmltodict.parse(res.text))

dictTemp = xmltodict.parse(res.text)

In [ ]:
# 필요한 값 추출
pd.DataFrame(dictTemp['response'])

In [ ]:
# 웹으로 OpenAPI url 호출하는 함수
def getRequestUrl(url):
    try:
        res = requests.get(url, verify= False)
        if res.status_code == 200: # 웹 URL 요청결과 OK
            return res.text # 웹으로 전달될 때는 Url, utf-8로 인코딩, 돌려받을 때는 utf-8 디코딩
    except Exception as e:
        print(f'getRequestUrl() 함수 호출 예외 : {e}')
        return None

In [ ]:
# OpenAPI 호출함수
def getTouristDataService(yymmdd):
    serviceKey = r'qD9EPBoeW3jjfktf3X%2Ftr3syHNjWfOzloUDZ5WfJwKtsyVCRzFy2v4OfNffjH5ulAtfuFrINMfhraiEFLeVmNQ%3D%3D'
    baseUrl = 'http://apis.data.go.kr/B551011/DataLabService/metcoRegnVisitrDDList?'
    parameters = 'numOfRows=60&pageNo=1&MobileOS=ETC&MobileApp=AppTest&'+\
                f'serviceKey={serviceKey}&startYmd={yymmdd}&endYmd={yymmdd}'
    
    totalUrl = baseUrl + parameters
    
    result = getRequestUrl(totalUrl)
    if result == None:
        return None
    else:
        return result

In [ ]:
# 실행테스트
data = getTouristDataService('20231202')

In [ ]:
# xml을 딕셔너리로 형변환후 필요한 데이터까지만 추출
xmltodict.parse(data)['response']['body']['items']['item']

In [ ]:
## 추가라이브러리 사용등록
from tqdm import tqdm
import time

In [ ]:
## 반복문으로 처리
touristDatas = []
year =2023
month = 12

for day in tqdm(range(1,32)):
    reqYmd = f'{year}{month}{day:02d}'
    data = getTouristDataService(reqYmd)
    if xmltodict.parse(data)['response']['body']['items'] == None:
        resList = []
    else:
        resList = xmltodict.parse(data)['response']['body']['items']['item']

    touristDatas = touristDatas+resList

In [ ]:
len(touristDatas)

In [ ]:
dfTourist = pd.DataFrame(touristDatas)

In [21]:
dfTourist.to_excel('./data/Tourist_dataportal.xlsx',index=False)